Добавляем в sys.path корнь проекта, чтобы нашлись все импорты (а можно было бы просто из корня нотбук запускать)

In [1]:
import sys
import os

project_root = os.path.abspath('..')

if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

# Импортируем классы моделей из нашего файла
from models import TransformerClassifier, MambaClassifier, LSTMClassifier


/Users/gab1k/Desktop/monkey_coding_dl_project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_TO_TRAIN = 'Transformer' 

# Гиперпараметры данных и модели
MAX_SEQ_LEN = 300
EMBEDDING_DIM = 128
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_EPOCHS = 5 # Увеличим для лучшего результата

# Пути для сохранения артефактов
SAVE_DIR = "pretrained"
os.makedirs(SAVE_DIR, exist_ok=True)
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, "best_model.pth")
VOCAB_SAVE_PATH = os.path.join(SAVE_DIR, "vocab.json")
CONFIG_SAVE_PATH = os.path.join(SAVE_DIR, "config.json")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
from data.preprocessor import DatasetGenerator, DatasetConfig
from data.datasets_params import DatasetName

generator = DatasetGenerator(DatasetName.IMDB)
(X_train, y_train), (X_val, y_val), (X_test, y_test) = generator.generate_dataset()
X_train, y_train, X_val, y_val, X_test, y_test = X_train[:1000], y_train[:1000], X_val[:100], y_val[:100], X_test[:100], y_test[:100]

In [5]:
def create_dataloader(X, y, batch_size):
    dataset = TensorDataset(torch.tensor(X, dtype=torch.long), torch.tensor(y, dtype=torch.long))
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)
train_loader = create_dataloader(X_train, y_train, BATCH_SIZE)
val_loader = create_dataloader(X_val, y_val, BATCH_SIZE)

In [6]:
model_params = {}
if MODEL_TO_TRAIN == 'Transformer':
    model_params = {'vocab_size': len(generator.vocab), 'embed_dim': EMBEDDING_DIM, 'num_heads': 4, 'num_layers': 2, 'num_classes': 2, 'max_seq_len': MAX_SEQ_LEN}
    model = TransformerClassifier(**model_params)
elif MODEL_TO_TRAIN == 'Mamba':
    model_params = {'vocab_size': len(generator.vocab), 'embed_dim': EMBEDDING_DIM, 'mamba_d_state': 16, 'mamba_d_conv': 4, 'mamba_expand': 2, 'num_classes': 2}
    model = MambaClassifier(**model_params)
elif MODEL_TO_TRAIN == 'LSTM':
    model_params = {'vocab_size': len(generator.vocab), 'embed_dim': EMBEDDING_DIM, 'hidden_dim': 256, 'num_layers': 2, 'num_classes': 2, 'dropout': 0.5}
    model = LSTMClassifier(**model_params)
else:
    raise ValueError("Неизвестный тип модели. Выберите 'Transformer', 'Mamba' или 'LSTM'")

In [7]:
model.to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

best_val_loss = float('inf')
print(f"--- Начало обучения модели: {MODEL_TO_TRAIN} ---")
for epoch in range(NUM_EPOCHS):
    model.train()
    start_time = time.time()
    total_train_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(DEVICE), batch_y.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(train_loader)
    
    model.eval()
    total_val_loss, correct_val, total_val = 0, 0, 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(DEVICE), batch_y.to(DEVICE)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += batch_y.size(0)
            correct_val += (predicted == batch_y).sum().item()
    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = correct_val / total_val

    epoch_time = time.time() - start_time
    print(f"Эпоха {epoch+1}/{NUM_EPOCHS} | Время: {epoch_time:.2f}с | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")
    
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"  -> Модель сохранена, новая лучшая Val Loss: {best_val_loss:.4f}")

--- Начало обучения модели: Transformer ---
Эпоха 1/5 | Время: 8.49с | Train Loss: 0.6953 | Val Loss: 0.6727 | Val Acc: 0.5900
  -> Модель сохранена, новая лучшая Val Loss: 0.6727
Эпоха 2/5 | Время: 8.39с | Train Loss: 0.6722 | Val Loss: 0.6586 | Val Acc: 0.6000
  -> Модель сохранена, новая лучшая Val Loss: 0.6586
Эпоха 3/5 | Время: 8.60с | Train Loss: 0.6516 | Val Loss: 0.6720 | Val Acc: 0.6800
Эпоха 4/5 | Время: 8.59с | Train Loss: 0.6329 | Val Loss: 0.6965 | Val Acc: 0.5100
Эпоха 5/5 | Время: 8.62с | Train Loss: 0.6088 | Val Loss: 0.6719 | Val Acc: 0.7000


In [9]:
with open(VOCAB_SAVE_PATH, 'w', encoding='utf-8') as f:
    json.dump(generator.vocab, f, ensure_ascii=False, indent=4)

config = {
    "model_type": MODEL_TO_TRAIN,
    "max_seq_len": MAX_SEQ_LEN,
    "model_params": model_params,
}
with open(CONFIG_SAVE_PATH, 'w', encoding='utf-8') as f:
    json.dump(config, f, ensure_ascii=False, indent=4)
print(f"Конфигурация модели сохранена в: {CONFIG_SAVE_PATH}")


Конфигурация модели сохранена в: pretrained/config.json
